In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import utils
from utils import *
import importlib
import argparse
import trader   # 파일 수정 반영
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
binance = get_binance()

In [37]:
tf = "1m"
limit = 1500
past_df_dic = {}
new_symlist = []
for sym in SYMLIST[0:50]:
    vol = await binance.fetch_tickers(symbols=[sym])
    if (not len(list(vol.values())) > 0) or list(vol.values())[0]['quoteVolume'] < 20*(10**6):
        continue
    df = await past_data(binance, sym, tf, limit)
    df["mean"] = np.mean([df["open"], df["high"], df["low"], df["close"]], axis=0)
    past_df_dic.update({sym: df})
    new_symlist.append(sym)

In [23]:
def bull_or_bear(df,ref=0.05):
    m = (df['high']+df['low'])/2  # df['close']로 해도 되는데 그냥 이렇게 함
    rising = []
    for i in range(1, len(m)-1):
        rising.append((m[i+1] - m[i])/m[i]*100)
    rising_coef = np.mean(rising)
    if rising_coef > ref:
        return "BULL", rising_coef
    elif rising_coef < -ref:
        return "BEAR", rising_coef
    else:
        return "~-~-~", rising_coef
    
def inspect_market(df, print_=True):
    st1, score1 = bull_or_bear(df[:10])
    st2, score2 = bull_or_bear(df)
    score = score1+score2
    if st1 == 'BEAR' and st2 == "BEAR":
        return SHORT, score
    elif st1 == 'BULL' and st2 == "BULL":         # 4시간동안 상승
        return LONG, score
    return None, 0

In [204]:
def select_sym(t):
    max_score, min_score = 0,0
    max_sym, min_sym = 0,0
    for sym in new_symlist:
        df = past_df_dic[sym][t-20:t]
        position, score = inspect_market(df)
        
        if position == LONG and score > max_score:
            max_score, max_sym = score, sym
        elif position == SHORT and score < min_score:
            min_score, min_sym = score, sym

    if max_score > 0 or min_score < 0:
        print(f"== MAX: {max_sym} {max_score} | MIN: {min_sym} {min_score} ==")
        if abs(max_score) > abs(min_score):
            return max_sym, SHORT
        else:
            return min_sym, LONG 
    return None, None

In [205]:

def get_pnl(open_price, price):
    return (price - open_price)/open_price

def run(t, sl, tp, lev):
    sym, position = select_sym(t)
    if not sym and not position:
        return 0,0,0,0,0,0
    sym_df = past_df_dic[sym]['mean'][t:]

    open_price = sym_df[0]
    
    for i, mean_price in enumerate(sym_df[1:]):
        coef = 1 if position == "Long" else -1
        pnl = get_pnl(open_price, mean_price) * lev * coef * 100 - 0.7
        if pnl > tp or pnl < sl:
            return i+1, open_price, mean_price, pnl, sym, position
        
    return 0,0,0,0,0,0
        

def show_default_graph(df, t, time_i, open_price, close_price, position, title=""):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize = (12,6))

    x = len(df)
    
    ax.plot(range(x), df, color = (1, 0.7, 0), linewidth = 2, label='mean price')
        
    color = 'b' if position == 'Short' else 'r'
    ax.add_patch(plt.Arrow(
        t, open_price, time_i, close_price-open_price,
        width = (close_price-open_price)*2, color=color, zorder=2, alpha=0.5
    ))
        
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')

    ax.set_title(title)
    plt.savefig(f"backtest_result/{t}.jpg")
    return ax

def simul(pnls):
    orig = 100
    for pnl in pnls:
        if pnl != 0:
            orig *= (1+pnl*0.01)
    return orig

In [208]:
sl, tp = -10, 10
t = 20
lev = 10
pnls = []
episode = 10

for t in range(20, 1400, 10):
    time_i, open_price, close_price, pnl, sym, position = run(t, sl, tp, lev)
    pnls.append(pnl)
    # if pnl != 0:
    #     show_default_graph(past_df_dic[sym]['mean'], t, time_i, open_price, close_price, position, title=f"{sym}  PNL: {pnl}%")
print(np.mean(pnls))

== MAX: 0 0 | MIN: KEY/USDT -0.36286534294927486 ==
== MAX: 0 0 | MIN: KEY/USDT -0.26142280652987476 ==
== MAX: 0 0 | MIN: BEAMX/USDT -0.21932519323036115 ==
== MAX: 0 0 | MIN: KEY/USDT -0.23620871932260867 ==
== MAX: 0 0 | MIN: TIA/USDT -0.21795081151675244 ==
== MAX: 0 0 | MIN: ORDI/USDT -0.22653542574711266 ==
== MAX: AXL/USDT 0.2663992580531277 | MIN: KEY/USDT -0.561414331427087 ==
== MAX: AXL/USDT 0.24998609510520886 | MIN: KEY/USDT -0.14838454230409814 ==
== MAX: TIA/USDT 0.2672073611564185 | MIN: 0 0 ==
== MAX: TIA/USDT 0.18365097195840818 | MIN: 0 0 ==
== MAX: AXL/USDT 0.20704493174583577 | MIN: 0 0 ==
== MAX: KEY/USDT 0.34022924224962336 | MIN: AXL/USDT -0.11974235281521793 ==
== MAX: AXL/USDT 0.13527742974209417 | MIN: 0 0 ==
== MAX: SXP/USDT 0.3091138305347294 | MIN: 0 0 ==
== MAX: SXP/USDT 0.4078899518419965 | MIN: 0 0 ==
== MAX: 0 0 | MIN: MANTA/USDT -0.20626803898696638 ==
== MAX: 0 0 | MIN: MANTA/USDT -0.25749962986495467 ==
== MAX: 0 0 | MIN: AXL/USDT -0.136469428989383

In [209]:
pnls = np.array(pnls)
print(f"Total PNL: {np.sum(pnls)}%,    # of good pred: {np.sum(np.where(pnls>0, 1, 0))},    bad pred: {np.sum(np.where(pnls<0, 1, 0))}")
simul(pnls)

Total PNL: 24.408038233073675%,    # of good pred: 39,    bad pred: 36


76.76315506363336